In [ ]:
import json
from pathlib import Path

import numpy as np
import torch

try:
    from jupythree.pointcloud import pointcloud
except ImportError:
    %pip install jupythree
    from jupythree.pointcloud import pointcloud

In [ ]:
DATA_DIR = Path("/datasets/mmscan_data/embodiedscan_split/embodiedscan-v1")

# load annotations
with open(DATA_DIR / "embodiedscan_train_vg_all_filtered.json") as f:
    anno_all = json.load(f)

In [ ]:
for sample in anno_all:
    if "matterport3d" in sample["scan_id"]:
        print(sample["scan_id"])
        break

for sample in anno_all:
    if "3rscan" in sample["scan_id"]:
        print(sample["scan_id"])
        break

for sample in anno_all:
    if "scannet" in sample["scan_id"]:
        print(sample["scan_id"])
        break

In [ ]:
with open("../src/data/embodiedscan/meta_data/mp3d_mapping.json") as f:
    mp3d_mapping = json.load(f)

In [ ]:
def visualize_sample(dataset: str, scene_id: str):

    if dataset == "matterport3d":
        scene, room = scene_id.split("_")
        scene_mapped = mp3d_mapping[scene]
        scene_id = f"{scene_mapped}_{room}"

    print(scene_id)

    # load pcd
    pcd, color, label, instance_ids = torch.load(DATA_DIR / "process_pcd" / f"{scene_id}.pth")
    # jupythree axis
    pcd[..., [1, 2]] = pcd[..., [2, 1]]

    print(
        f"pcd: {pcd.shape}, color: {color.shape}, label: {label.shape}, instance_ids: {instance_ids.shape}"
    )

    anno_scene = []
    for sample in anno_all:
        if dataset == "matterport3d":
            if sample["scan_id"] == f"matterport3d/{scene}/{room}":
                anno_scene.append(sample)
        else:
            if sample["scan_id"] == f"{dataset}/{scene_id}":
                anno_scene.append(sample)

    print(f"# annotations for {scene_id}: {len(anno_scene)}")

    anno_idx = 0
    anno = anno_scene[anno_idx]

    instance_id = anno["target_id"]
    instance_id_mask = instance_ids == instance_id
    query = anno["text"]

    print(f"query: {query}")

    color_masked = color.copy()
    color_masked[instance_id_mask] = [1, 0, 0]

    return pointcloud(pcd, color=color_masked, radius=2).show(width=1000)


def visualize_coverage(dataset: str, scene_id: str):
    if dataset == "matterport3d":
        scene, room = scene_id.split("_")
        scene_mapped = mp3d_mapping[scene]
        scene_id = f"{scene_mapped}_{room}"

    print(scene_id)

    # load pcd
    pcd, _, _, instance_ids = torch.load(DATA_DIR / "process_pcd" / f"{scene_id}.pth")
    # jupythree axis
    pcd[..., [1, 2]] = pcd[..., [2, 1]]

    anno_scene = []
    for sample in anno_all:
        if dataset == "matterport3d":
            if sample["scan_id"] == f"matterport3d/{scene}/{room}":
                anno_scene.append(sample)
        else:
            if sample["scan_id"] == f"{dataset}/{scene_id}":
                anno_scene.append(sample)

    print(f"# annotations for {scene_id}: {len(anno_scene)}")

    mask = np.zeros(len(pcd), dtype=bool)
    for anno in anno_scene:
        instance_id = anno["target_id"]
        if instance_id == -100:
            continue

        instance_id_mask = instance_ids == instance_id
        mask[instance_id_mask] = 1

    color_masked = np.zeros_like(pcd)
    color_masked[mask == 1] = [1, 0, 0]

    return pointcloud(pcd, color=color_masked, radius=2).show(width=1000)

## ScanNet

In [ ]:
# embodiedscan
pcd, _, _, _ = torch.load(DATA_DIR / "process_pcd" / "scene0000_00.pth")
center = pcd.mean(axis=0)
center[2] = 0
pcd = pcd - center
print(len(pcd))

# scannet
coord = np.load("/datasets/scannet_hf/train/scene0000_00/coord.npy")
center2 = coord.mean(axis=0)
center2[2] = 0
coord = coord - center2
print(len(coord))

# concatenated
pcd_all = np.concatenate([pcd, coord], axis=0)
color = np.zeros_like(pcd_all)
color[: len(pcd)] = [1, 0, 0]
color[len(pcd) :] = [0, 1, 0]

pointcloud(pcd_all, color=color, radius=2).show(width=1000)

In [ ]:
# visualize_sample("scannet", "scene0000_00")
visualize_coverage("scannet", "scene0103_01")

## Matterport3D

In [ ]:
visualize_sample("matterport3d", "ZMojNkEp431_region0")
# visualize_coverage("matterport3d", "ZMojNkEp431_region0")

## 3RScan

In [ ]:
visualize_sample("3rscan", "fcf66d9e-622d-291c-84c2-bb23dfe31327")